In [1]:
import pandas as pd
import scanpy as sc
import anndata as ad

/tmp/ipykernel_4129776/1891703346.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dir = '/Genomics/pritykinlab/zhiyue/557/data/'
adata = sc.read_h5ad(dir + 'Integrated_normalized_w_metadata.h5ad')
adata

AnnData object with n_obs × n_vars = 224611 × 2917
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Percent_mt', 'Patient', 'Gender', 'Subtype', 'Stage', 'seurat_clusters', 'Study', 'Cell_Cluster_level1', 'Cell_Cluster_level2', 'validation_original_annotation', 'id', 'predicted.celltypel1.score', 'predicted.celltypel1', 'predicted.celltypel2.score', 'predicted.celltypel2'
    var: 'gene_name'

### This anndata contains 224611 observations (cells) and 2917 variables (genes). More info on anndata: https://anndata.readthedocs.io/en/latest/

In [4]:
# X is the raw counts
adata.X

<224611x2917 sparse matrix of type '<class 'numpy.float32'>'
	with 274163106 stored elements in Compressed Sparse Row format>

In [3]:
# obs contains cell information
adata.obs
# We are most interested in 'Cell_Cluster_level1' and 'Cell_Cluster_level2' which we want to predict; these columns are also in cell_type_labels.csv
# 'id' tells you if it's reference or validation

,orig.ident,nCount_RNA,nFeature_RNA,Percent_mt,Patient,Gender,Subtype,Stage,seurat_clusters,Study,Cell_Cluster_level1,Cell_Cluster_level2,validation_original_annotation,id,predicted.celltypel1.score,predicted.celltypel1,predicted.celltypel2.score,predicted.celltypel2
P1_X1_TCCCATTCTCCT,X1,6872.0,421,1.309662,S5P1,F,squamous cell carcinoma,III/IV,0,GSE148071,T,Naive T,NaN,Reference,NaN,NaN,NaN,NaN
P1_X1_AGCTCCGGCACC,X1,12658.0,2512,6.367515,S5P1,F,squamous cell carcinoma,III/IV,10,GSE148071,Cancer,CXCL1 Cancer,NaN,Reference,NaN,NaN,NaN,NaN
P1_X1_TACCATTACTTG,X1,11646.0,1679,9.222050,S5P1,F,squamous cell carcinoma,III/IV,6,GSE148071,Plasma,Plasma,NaN,Reference,NaN,NaN,NaN,NaN
P1_X1_TGTCTCGCATCC,X1,11038.0,1229,4.076826,S5P1,F,squamous cell carcinoma,III/IV,6,GSE148071,Plasma,Plasma,NaN,Reference,NaN,NaN,NaN,NaN
P1_X1_TAGACTACGTAG,X1,9671.0,1606,9.854203,S5P1,F,squamous cell carcinoma,III/IV,6,GSE148071,Plasma,Plasma,NaN,Reference,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p7t2_bcCRCT,SeuratProject,648.0,335,2.006173,S1P7,F,adenocarcinoma,III,13,GSE127465,Myeloid,Neutrophils,Neutrophils,Validation,1.0,Myeloid cells,1.000000,Neutrophils
p7t2_bcFAUI,SeuratProject,550.0,336,18.363636,S1P7,F,adenocarcinoma,III,16,GSE127465,Mast,Mast,Mast cells,Validation,1.0,Mast cells,1.000000,Mast cells
p7t2_bcFHTY,SeuratProject,549.0,367,5.828780,S1P7,F,adenocarcinoma,III,2,GSE127465,Myeloid,Lipid-associated Mac,Macrophages,Validation,1.0,Myeloid cells,0.698839,Lipid-associated moMac
p7t2_bcBHQQ,SeuratProject,429.0,295,6.060606,S1P7,F,adenocarcinoma,III,15,GSE127465,Myeloid,Low quality Mac,Macrophages,Validation,1.0,Myeloid cells,0.839702,Low quality Mac


In [5]:
# var contains gene informtion (so far only the gene names; we will only worry about this part if we include other datasets)
adata.var

,gene_name
0,IGHM
1,IGLC3
2,IGLC2
3,IGHA1
4,IGHG1
...,...
2912,SLC27A2
2913,ARRB1
2914,ZNF37A
2915,NCAPD2
